This Notebook is for Problem 9

Image classification with neural networks

In [9]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split


In [11]:
# Define transformations for the training and test sets
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64 pixels
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset from the EuroSAT_RGB directory
dataset = datasets.ImageFolder('EuroSAT_RGB', transform=transform)

# Split the dataset into training (80%) and testing (20%) sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [21]:
import torch.nn as nn
import torch.nn.functional as F


In [12]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 11)  # 11 classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = SimpleCNN()


In [20]:
from torch import optim
from tqdm import tqdm

In [13]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop with tqdm progress bar
n_epochs = 10
for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    # Use tqdm to create a progress bar for the current epoch
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{n_epochs}', ncols=80)
    
    for images, labels in progress_bar:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Update the progress bar description with the current loss and accuracy
        progress_bar.set_postfix({'Loss': running_loss/len(train_loader), 'Acc': f'{100*correct/total:.2f}%'})

    # Print epoch summary
    train_accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}/{n_epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%')


Epoch 1/10:   0%|                                       | 0/675 [00:00<?, ?it/s]

Epoch 1/10: 100%|█████| 675/675 [03:43<00:00,  3.03it/s, Loss=0.897, Acc=66.93%]


Epoch 1/10, Loss: 0.8969, Accuracy: 66.93%


Epoch 2/10: 100%|█████| 675/675 [02:45<00:00,  4.09it/s, Loss=0.493, Acc=82.34%]


Epoch 2/10, Loss: 0.4933, Accuracy: 82.34%


...

Epoch 8/10, Loss: 0.0896, Accuracy: 97.14%


Epoch 9/10: 100%|████| 675/675 [02:54<00:00,  3.88it/s, Loss=0.0787, Acc=97.38%]


Epoch 9/10, Loss: 0.0787, Accuracy: 97.38%


Epoch 10/10: 100%|███| 675/675 [02:49<00:00,  3.99it/s, Loss=0.0777, Acc=97.51%]

Epoch 10/10, Loss: 0.0777, Accuracy: 97.51%


Epoch 1/10: 100%|█████| 675/675 [03:43<00:00,  3.03it/s, Loss=0.897, Acc=66.93%]

Epoch 1/10, Loss: 0.8969, Accuracy: 66.93%

Epoch 2/10: 100%|█████| 675/675 [02:45<00:00,  4.09it/s, Loss=0.493, Acc=82.34%]

Epoch 2/10, Loss: 0.4933, Accuracy: 82.34%

Epoch 3/10: 100%|█████| 675/675 [02:34<00:00,  4.36it/s, Loss=0.368, Acc=87.05%]

Epoch 3/10, Loss: 0.3684, Accuracy: 87.05%

Epoch 4/10: 100%|█████| 675/675 [02:35<00:00,  4.34it/s, Loss=0.268, Acc=90.41%]

Epoch 4/10, Loss: 0.2676, Accuracy: 90.41%

Epoch 5/10: 100%|█████| 675/675 [03:10<00:00,  3.54it/s, Loss=0.194, Acc=93.22%]

Epoch 5/10, Loss: 0.1938, Accuracy: 93.22%

Epoch 6/10: 100%|█████| 675/675 [02:36<00:00,  4.30it/s, Loss=0.147, Acc=94.92%]

Epoch 6/10, Loss: 0.1468, Accuracy: 94.92%

Epoch 7/10: 100%|█████| 675/675 [02:35<00:00,  4.34it/s, Loss=0.113, Acc=96.30%]

Epoch 7/10, Loss: 0.1135, Accuracy: 96.30%

Epoch 8/10: 100%|████| 675/675 [02:35<00:00,  4.33it/s, Loss=0.0896, Acc=97.14%]

Epoch 8/10, Loss: 0.0896, Accuracy: 97.14%

Epoch 9/10: 100%|████| 675/675 [02:54<00:00,  3.88it/s, Loss=0.0787, Acc=97.38%]

Epoch 9/10, Loss: 0.0787, Accuracy: 97.38%

Epoch 10/10: 100%|███| 675/675 [02:49<00:00,  3.99it/s, Loss=0.0777, Acc=97.51%]

Epoch 10/10, Loss: 0.0777, Accuracy: 97.51%

In [22]:
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f'Test Set Accuracy: {accuracy * 100:.2f}%')

# confusion matrix
conf_matrix = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:")
print(conf_matrix)

# plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=dataset.classes, yticklabels=dataset.classes)
plt.xlabel('Predicted Label')
plt.ylabel('Actual Label')
plt.title('Confusion Matrix')

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix / np.sum(conf_matrix, axis=1, keepdims=True), annot=True, fmt='.2%', cmap='Blues', xticklabels=dataset.classes, yticklabels=dataset.classes)
plt.xlabel('Predicted Label')
plt.ylabel('Actual Label')
plt.title('Normalized Confusion Matrix')

Test Set Accuracy: 88.65%

Confusion Matrix:

![alt text](images/conf_matrix1.png)

![alt text](images/conf_matrix2.png)

![alt text](images/conf_matrix3.png)

In [ ]:
# first 10 images in test set
# predicted and actual values
for i in range(10):
    imshow(images[i], f'Predicted: {dataset.classes[preds[i]]}, Actual: {dataset.classes[labels[i]]}')


First 10 Predictions:

![alt text](images/predicted_1.png)

![alt text](images/predicted_2.png)

![alt text](images/predicted_3.png)

![alt text](images/predicted_4.png)

![alt text](images/predicted_5.png)

![alt text](images/predicted_6.png)

![alt text](images/predicted_7.png)

![alt text](images/predicted_8.png)

![alt text](images/predicted_9.png)

![alt text](images/predicted_10.png)

In [36]:
from sklearn.metrics import classification_report

In [ ]:
class_report = classification_report(all_labels, all_preds, target_names=dataset.classes)
print(class_report)

Classification Report:

![alt text](images/class_report.png)

In [44]:
topk = 3
model.eval()
correct_topk = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, topk_preds = torch.topk(outputs, topk, dim=1)
        correct_topk += (topk_preds == labels.view(-1, 1)).sum().item()
        total += labels.size(0)

topk_accuracy = 100 * correct_topk / total
print(f'Top-{topk} Accuracy: {topk_accuracy:.2f}%')


Top-3 Accuracy: 98.50%


Top-3 Accuracy: 98.50%

This means that 98.50% of the time, the correct class was within the top 3 predictions made by the model.

The model achieves a good accuracy score of 88.65%. The model is summed up through a confusion matrix (shown normalized and unnormalized), the printed results of the first 10 images in the testing set, a classification matrix, and a top-k accurracy calculation.

The model is best at identifying images in the classes of SeaLake, Residential, and Forest. It struggles the most with HerbaciousVegetaion, PermanentCrop, and River.